In [24]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [25]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    return_messages=True,
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

In [26]:
examples = [
    {
        "question": "Top Gun",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "question": "Godfather",
        "answer": "👨‍👨‍👦🔫🍝",
    },
    {
        "question": "The Avengers",
        "answer": "🛡️🔨👽",
    },
]

In [27]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "Print the answer for the movie you want in the {question} ONLY using 3 emojis."),
    ("ai", "{answer}"),
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a expert about movies."),
    MessagesPlaceholder(variable_name="history"),
    example_prompt,
    ("human", "{question}"),
])

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(question):
    result = chain.invoke({
        "question": question,
    })
    memory.save_context(
        {"input": question},
        {"output": result.content}
    )

In [28]:
invoke_chain("Titanic")

🚢💔🌊

In [29]:
invoke_chain("Avator")

🌿🌌💙

In [30]:
invoke_chain("What was the name of the movie I asked first?")

You asked about "Titanic."